# 성능튜닝

## 1.환경준비

### (1) import

In [1]:
#라이브러리들을 불러오자.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 전처리
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# 모델링
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import * 

import warnings    # 경고메시지 제외
warnings.filterwarnings(action='ignore')

### (2) 데이터 준비

* 변수설명
    * COLLEGE : 대학 졸업여부
    * INCOME : 연수입
    * OVERAGE : 월평균 초과사용 시간(분)
    * LEFTOVER : 월평균 잔여시간비율(%)
    * HOUSE : 집값
    * HANDSET_PRICE : 스마트폰 가격
    * OVER_15MINS_CALLS_PER_MONTH : 월평균 장기통화(15분이상) 횟수
    * AVERAGE_CALL_DURATION : 평균 통화 시간
    * REPORTED_SATISFACTION : 만족도 설문조사 결과
    * REPORTED_USAGE_LEVEL : 사용도 자가진단 결과
    * CONSIDERING_CHANGE_OF_PLAN : 향후 변경계획 설문조사 결과
    * CHURN : 이탈(번호이동) 여부 (1-이탈, 0-잔류, Target 변수)


In [2]:
# 데이터를 불러옵시다.
path = 'https://raw.githubusercontent.com/DA4BAM/dataset/master/mobile_cust_churn.csv'
data = pd.read_csv(path)
data = data.sample(5000, random_state = 2022)
data['CHURN'] = data['CHURN'].map({'LEAVE':1, 'STAY':0})
data.head()

,id,COLLEGE,INCOME,OVERAGE,LEFTOVER,HOUSE,HANDSET_PRICE,OVER_15MINS_CALLS_PER_MONTH,AVERAGE_CALL_DURATION,REPORTED_SATISFACTION,REPORTED_USAGE_LEVEL,CONSIDERING_CHANGE_OF_PLAN,CHURN
3178,3179,0,119512,51,31,248566,229,5,2,very_sat,very_high,considering,1
14926,14927,1,142144,192,15,774317,581,29,4,unsat,very_little,never_thought,1
15116,15117,1,142308,0,79,306426,497,1,1,sat,little,considering,0
12733,12734,1,113385,0,0,333599,819,1,6,very_unsat,very_high,considering,1
14032,14033,1,90348,209,10,637286,360,26,4,unsat,little,actively_looking_into_it,0


## 2.데이터 준비

### (1) 데이터 정리

In [3]:
drop_cols = ['id']
data.drop(drop_cols, axis = 1, inplace = True )

### (2) 데이터분할1 : x, y 나누기

In [4]:
target = 'CHURN'
x = data.drop(target, axis = 1)
y = data.loc[:, target]

### (3) NA 조치

### (4) 가변수화

In [5]:
dumm_cols = ['REPORTED_SATISFACTION','REPORTED_USAGE_LEVEL','CONSIDERING_CHANGE_OF_PLAN']
x = pd.get_dummies(x, columns = dumm_cols, drop_first = True)

### (5) 데이터분할2 : train : validation 나누기

In [6]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = .3, random_state = 20)

### (6) Scaling

In [7]:
scaler = MinMaxScaler()
x_train_s = scaler.fit_transform(x_train)
x_val_s = scaler.transform(x_val)

## 3.선형모델 튜닝

Logistic Regression : 전진선택법
* 변수를 하나씩 늘려가면서
* AIC를 가장 낮추는 모델 찾기

### (1) 전진선택을 수행할 함수 만들기( **로지스틱 회귀** 용)

In [8]:
# 아래 함수는 로지스틱 회귀를 위한 전진선택법 함수 입니다.
import statsmodels.api as sm

def forward_stepwise_logistic(x_train, y_train):

    # 변수목록, 선택된 변수 목록, 단계별 모델과 AIC 저장소 정의
    features = list(x_train)
    selected = []
    step_df = pd.DataFrame({ 'step':[], 'feature':[],'aic':[]})

    # 
    for s in range(0, len(features)) :
        result =  { 'step':[], 'feature':[],'aic':[]}

        # 변수 목록에서 변수 한개씩 뽑아서 모델에 추가
        for f in features :
            vars = selected + [f]
            x_tr = x_train[vars]
            model = sm.Logit(y_train, x_tr).fit()
            result['step'].append(s+1)
            result['feature'].append(vars)
            result['aic'].append(model.aic)
        
        # 모델별 aic 집계
        temp = pd.DataFrame(result).sort_values('aic').reset_index(drop = True)

        # 만약 이전 aic보다 새로운 aic 가 크다면 멈추기
        if step_df['aic'].min() < temp['aic'].min() :
            break
        step_df = pd.concat([step_df, temp], axis = 0).reset_index(drop = True)

        # 선택된 변수 제거
        v = temp.loc[0,'feature'][s]
        features.remove(v)

        selected.append(v)
    
    # 선택된 변수와 step_df 결과 반환
    return selected, step_df

### (2) 전진선택법 수행

In [9]:
vars, result = forward_stepwise_logistic(x_train, y_train)

Optimization terminated successfully.
         Current function value: 0.693075
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.693062
         Iterations 2
Optimization terminated successfully.
         Current function value: 0.683528
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.693007
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.684909
         Iterations 2
Optimization terminated successfully.
         Current function value: 0.693042
         Iterations 2
Optimization terminated successfully.
         Current function value: 0.686899
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.692565
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692806
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.693137
  

Optimization terminated successfully.
         Current function value: 0.632641
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.633102
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.632954
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.632697
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.633174
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.633137
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.633162
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.633092
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.633175
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.633193
  

* 선택된 변수

In [10]:
vars

['OVERAGE',
 'HOUSE',
 'HANDSET_PRICE',
 'LEFTOVER',
 'REPORTED_SATISFACTION_very_sat',
 'INCOME',
 'REPORTED_SATISFACTION_sat']

In [11]:
result

,step,feature,aic
0,1.0,[OVERAGE],4786.699456
1,1.0,[HOUSE],4796.363859
2,1.0,[OVER_15MINS_CALLS_PER_MONTH],4810.294604
3,1.0,[REPORTED_SATISFACTION_very_sat],4845.064834
4,1.0,[AVERAGE_CALL_DURATION],4849.951663
...,...,...,...
114,7.0,"[OVERAGE, HOUSE, HANDSET_PRICE, LEFTOVER, REPO...",4442.309949
115,7.0,"[OVERAGE, HOUSE, HANDSET_PRICE, LEFTOVER, REPO...",4442.315315
116,7.0,"[OVERAGE, HOUSE, HANDSET_PRICE, LEFTOVER, REPO...",4442.316816
117,7.0,"[OVERAGE, HOUSE, HANDSET_PRICE, LEFTOVER, REPO...",4442.448187


In [12]:
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

In [13]:
result.sort_values(by='aic', ascending=True)

,step,feature,aic
105,7.0,"[OVERAGE, HOUSE, HANDSET_PRICE, LEFTOVER, REPORTED_SATISFACTION_very_sat, INCOME, REPORTED_SATISFACTION_sat]",4438.689496
90,6.0,"[OVERAGE, HOUSE, HANDSET_PRICE, LEFTOVER, REPORTED_SATISFACTION_very_sat, INCOME]",4440.486653
91,6.0,"[OVERAGE, HOUSE, HANDSET_PRICE, LEFTOVER, REPORTED_SATISFACTION_very_sat, REPORTED_SATISFACTION_sat]",4440.881058
106,7.0,"[OVERAGE, HOUSE, HANDSET_PRICE, LEFTOVER, REPORTED_SATISFACTION_very_sat, INCOME, CONSIDERING_CHANGE_OF_PLAN_considering]",4440.991703
107,7.0,"[OVERAGE, HOUSE, HANDSET_PRICE, LEFTOVER, REPORTED_SATISFACTION_very_sat, INCOME, REPORTED_USAGE_LEVEL_little]",4441.637064
...,...,...,...
15,1.0,[COLLEGE],4853.522912
16,1.0,[CONSIDERING_CHANGE_OF_PLAN_no],4853.560645
17,1.0,[REPORTED_SATISFACTION_unsat],4853.957884
18,1.0,[REPORTED_SATISFACTION_very_unsat],4854.023812


### (3) 모델링

* 전체 변수 

In [14]:
m1 = LogisticRegression()
m1.fit(x_train, y_train)
p1 = m1.predict(x_val)

print(accuracy_score(y_val, p1))
print(classification_report(y_val, p1))

0.6333333333333333
              precision    recall  f1-score   support

           0       0.62      0.68      0.65       738
           1       0.65      0.59      0.62       762

    accuracy                           0.63      1500
   macro avg       0.63      0.63      0.63      1500
weighted avg       0.64      0.63      0.63      1500



* 전진선택법 변수

In [15]:
m2 = LogisticRegression()
m2.fit(x_train[vars], y_train)
p2 = m2.predict(x_val[vars])

print(accuracy_score(y_val, p2))
print(classification_report(y_val, p2))

0.634
              precision    recall  f1-score   support

           0       0.62      0.68      0.65       738
           1       0.66      0.59      0.62       762

    accuracy                           0.63      1500
   macro avg       0.64      0.63      0.63      1500
weighted avg       0.64      0.63      0.63      1500



## 4.하이퍼파라미터 튜닝

### (1) 필요한 함수 불러오기 

In [16]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

### (2) Random Search

① 값의 범위를 지정한다.  
② 모델 선언(시도 횟수 지정)  
③ 모델링(값의 범위 내에서 시도 횟수만큼 랜덤하게 선택해서 시도한다.)  
④ 가장 성능이 좋은 값을 선정


#### ① 값의 범위를 지정한다.

In [17]:
# dictionary형태로 선언
params = { 'n_neighbors' : range(1,51), 'metric' : ['euclidean', 'manhattan']  }
params

{'n_neighbors': range(1, 51), 'metric': ['euclidean', 'manhattan']}

#### ② 모델 선언

In [18]:
# 기본모델
model = KNeighborsClassifier()

# Random Search 설정.
model_rs = RandomizedSearchCV(model
                            , params              # hyperparameter 범위 지정.
                            , cv=5                    # k-fold Cross Validation
                            , n_iter=5                # Random하게 시도할 횟수
                            )

#### ③ 모델링

In [19]:
# 학습 : model이 아니라 model_rs
model_rs.fit(x_train_s, y_train)

RandomizedSearchCV(cv=5, estimator=KNeighborsClassifier(), n_iter=5,
                   param_distributions={'metric': ['euclidean', 'manhattan'],
                                        'n_neighbors': range(1, 51)})

In [20]:
# 튜닝 결과
model_rs.cv_results_
# params와 mean_test_score을 참고할 것.

{'mean_fit_time': array([0.00103951, 0.0008009 , 0.00100098, 0.00080128, 0.00100079]),
 'std_fit_time': array([6.95528922e-04, 4.00447931e-04, 3.23406696e-07, 4.00638637e-04,
        1.16800773e-07]),
 'mean_score_time': array([0.0534483 , 0.05784817, 0.05845308, 0.05850205, 0.06545982]),
 'std_score_time': array([0.00257923, 0.00096377, 0.00196155, 0.00230598, 0.0035587 ]),
 'param_n_neighbors': masked_array(data=[3, 30, 24, 19, 4],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_metric': masked_array(data=['euclidean', 'euclidean', 'euclidean', 'euclidean',
                    'manhattan'],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'n_neighbors': 3, 'metric': 'euclidean'},
  {'n_neighbors': 30, 'metric': 'euclidean'},
  {'n_neighbors': 24, 'metric': 'euclidean'},
  {'n_neighbors': 19, 'metric': 'euclidean'},
  {'n_neighbors': 4, 'metric': 

In [21]:
model_rs.cv_results_['params']

[{'n_neighbors': 3, 'metric': 'euclidean'},
 {'n_neighbors': 30, 'metric': 'euclidean'},
 {'n_neighbors': 24, 'metric': 'euclidean'},
 {'n_neighbors': 19, 'metric': 'euclidean'},
 {'n_neighbors': 4, 'metric': 'manhattan'}]

In [22]:
model_rs.cv_results_['mean_test_score']

array([0.57885714, 0.584     , 0.58371429, 0.57914286, 0.58228571])

In [23]:
# 최적의 파라미터
model_rs.best_params_

{'n_neighbors': 30, 'metric': 'euclidean'}

In [24]:
# 그때의 성능
model_rs.best_score_

0.584

In [25]:
# best 모델로 예측 및 평가
pred = model_rs.predict(x_val_s)
print(classification_report(y_val, pred))

              precision    recall  f1-score   support

           0       0.55      0.74      0.63       738
           1       0.62      0.41      0.49       762

    accuracy                           0.57      1500
   macro avg       0.58      0.57      0.56      1500
weighted avg       0.58      0.57      0.56      1500



### (3) 실습 : Random Search

* decision tree로 튜닝을 시도해 봅시다.
    * max_depth : 1~10
    * min_samples_leaf : 10 ~ 100

#### ① 값의 범위를 지정한다.

In [26]:
params1 = {'max_depth': range(1,11)}
params2 = {'min_samples_leaf' : range(10,101)}
params = {'max_depth': range(1,11), 'min_samples_leaf' : range(10,101)}

#### ② 모델 선언

In [27]:
model = DecisionTreeClassifier()

model_rs1 = RandomizedSearchCV(model, params1, cv=5, n_iter=8)
model_rs2 = RandomizedSearchCV(model, params2, cv=5, n_iter=8)
model_rs = RandomizedSearchCV(model, params, cv=5, n_iter=10)

#### ③ 모델링

In [28]:
model_rs1.fit(x_train, y_train)
model_rs2.fit(x_train, y_train)
model_rs.fit(x_train, y_train)

RandomizedSearchCV(cv=5, estimator=DecisionTreeClassifier(),
                   param_distributions={'max_depth': range(1, 11),
                                        'min_samples_leaf': range(10, 101)})

In [29]:
# 최적의 파라미터
print(model_rs1.best_params_)
print(model_rs2.best_params_)
print(model_rs.best_params_)

{'max_depth': 3}
{'min_samples_leaf': 46}
{'min_samples_leaf': 34, 'max_depth': 8}


In [30]:
print(model_rs1.best_score_)
print(model_rs2.best_score_)
print(model_rs.best_score_)

0.692
0.6948571428571428
0.6994285714285715


In [31]:
# best 모델로 예측 및 평가
pred1= model_rs1.predict(x_val)
print(classification_report(y_val, pred1))
pred2= model_rs2.predict(x_val)
print(classification_report(y_val, pred2))
pred3= model_rs.predict(x_val)
print(classification_report(y_val, pred3))

              precision    recall  f1-score   support

           0       0.65      0.77      0.71       738
           1       0.73      0.60      0.66       762

    accuracy                           0.69      1500
   macro avg       0.69      0.69      0.68      1500
weighted avg       0.69      0.69      0.68      1500

              precision    recall  f1-score   support

           0       0.67      0.71      0.69       738
           1       0.70      0.67      0.68       762

    accuracy                           0.69      1500
   macro avg       0.69      0.69      0.69      1500
weighted avg       0.69      0.69      0.69      1500

              precision    recall  f1-score   support

           0       0.67      0.69      0.68       738
           1       0.69      0.68      0.69       762

    accuracy                           0.68      1500
   macro avg       0.68      0.68      0.68      1500
weighted avg       0.68      0.68      0.68      1500



### (4) Grid Search

① 값의 범위를 지정한다.  
② 모델링(값의 범위 내에서 모든 조합을 다 시도한다.)  
③ 가장 성능이 좋은 값을 선정


#### ① 값의 범위를 지정한다.

In [32]:
# dictionary형태로 선언
params = { 'n_neighbors' : range(3,31,2), 'metric' : ['euclidean', 'manhattan']  }
params

{'n_neighbors': range(3, 31, 2), 'metric': ['euclidean', 'manhattan']}

#### ② 모델 선언

In [33]:
# 기본모델
model = KNeighborsClassifier()

# Random Search 설정.
model_gs = GridSearchCV(model, params, cv=5)

#### ③ 모델링

In [34]:
# 학습 : model이 아니라 model_rs
model_gs.fit(x_train_s, y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'metric': ['euclidean', 'manhattan'],
                         'n_neighbors': range(3, 31, 2)})

In [35]:
# 튜닝 결과
model_gs.cv_results_

{'mean_fit_time': array([0.00100093, 0.00100136, 0.00100088, 0.00100083, 0.00100579,
        0.00060081, 0.00080094, 0.00080094, 0.00060058, 0.00040045,
        0.0008009 , 0.00100098, 0.00100102, 0.00100131, 0.00080075,
        0.00080113, 0.00060077, 0.0008007 , 0.0008009 , 0.00100121,
        0.00060091, 0.00100131, 0.0008007 , 0.00060706, 0.00060077,
        0.00100107, 0.00100093, 0.00080047]),
 'std_fit_time': array([2.78041453e-07, 3.01578299e-07, 4.52367448e-07, 1.78416128e-07,
        9.70993142e-06, 4.90563408e-04, 4.00471755e-04, 4.00471982e-04,
        4.90368586e-04, 4.90446468e-04, 4.00447931e-04, 8.60951905e-07,
        1.90734863e-07, 4.10190833e-07, 4.00376387e-04, 4.00567122e-04,
        4.90524413e-04, 4.00352620e-04, 4.00448413e-04, 6.32595976e-07,
        4.90641352e-04, 9.58430826e-07, 4.00353018e-04, 4.95819886e-04,
        4.90524366e-04, 7.29420592e-07, 9.93378957e-07, 4.00233365e-04]),
 'mean_score_time': array([0.04704266, 0.05695276, 0.05889277, 0.05780044, 

In [36]:
model_gs.cv_results_['params']

[{'metric': 'euclidean', 'n_neighbors': 3},
 {'metric': 'euclidean', 'n_neighbors': 5},
 {'metric': 'euclidean', 'n_neighbors': 7},
 {'metric': 'euclidean', 'n_neighbors': 9},
 {'metric': 'euclidean', 'n_neighbors': 11},
 {'metric': 'euclidean', 'n_neighbors': 13},
 {'metric': 'euclidean', 'n_neighbors': 15},
 {'metric': 'euclidean', 'n_neighbors': 17},
 {'metric': 'euclidean', 'n_neighbors': 19},
 {'metric': 'euclidean', 'n_neighbors': 21},
 {'metric': 'euclidean', 'n_neighbors': 23},
 {'metric': 'euclidean', 'n_neighbors': 25},
 {'metric': 'euclidean', 'n_neighbors': 27},
 {'metric': 'euclidean', 'n_neighbors': 29},
 {'metric': 'manhattan', 'n_neighbors': 3},
 {'metric': 'manhattan', 'n_neighbors': 5},
 {'metric': 'manhattan', 'n_neighbors': 7},
 {'metric': 'manhattan', 'n_neighbors': 9},
 {'metric': 'manhattan', 'n_neighbors': 11},
 {'metric': 'manhattan', 'n_neighbors': 13},
 {'metric': 'manhattan', 'n_neighbors': 15},
 {'metric': 'manhattan', 'n_neighbors': 17},
 {'metric': 'manha

In [37]:
model_gs.cv_results_['mean_test_score']

array([0.57885714, 0.57942857, 0.57714286, 0.57342857, 0.568     ,
       0.57457143, 0.58085714, 0.58257143, 0.57914286, 0.58057143,
       0.586     , 0.58457143, 0.57971429, 0.58028571, 0.57314286,
       0.59142857, 0.59171429, 0.59142857, 0.59828571, 0.59771429,
       0.59942857, 0.60142857, 0.60171429, 0.61057143, 0.61      ,
       0.61142857, 0.61857143, 0.618     ])

In [38]:
# 최적의 파라미터
model_gs.best_params_

{'metric': 'manhattan', 'n_neighbors': 27}

In [39]:
# 그때의 성능
model_gs.best_score_

0.6185714285714285

In [40]:
# best 모델로 예측 및 평가
pred = model_gs.predict(x_val_s)
print(classification_report(y_val, pred))

              precision    recall  f1-score   support

           0       0.59      0.72      0.65       738
           1       0.65      0.51      0.57       762

    accuracy                           0.61      1500
   macro avg       0.62      0.61      0.61      1500
weighted avg       0.62      0.61      0.61      1500



### (5) 실습 : Grid Search

* decision tree로 튜닝을 시도해 봅시다.
    * max_depth : 1~10
    * min_samples_leaf : 10 ~ 100

#### ① 값의 범위를 지정한다.

In [44]:
params1 = {'max_depth' : range(1,11), 'min_samples_leaf': range(10,101,5)}

#### ② 모델 선언

In [45]:
model1 = DecisionTreeClassifier()

model1_gs = GridSearchCV(model1, params1, cv=5)

#### ③ 모델링

In [46]:
model1_gs.fit(x_train, y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': range(1, 11),
                         'min_samples_leaf': range(10, 101, 5)})

In [47]:
model1_gs.cv_results_['params']

[{'max_depth': 1, 'min_samples_leaf': 10},
 {'max_depth': 1, 'min_samples_leaf': 15},
 {'max_depth': 1, 'min_samples_leaf': 20},
 {'max_depth': 1, 'min_samples_leaf': 25},
 {'max_depth': 1, 'min_samples_leaf': 30},
 {'max_depth': 1, 'min_samples_leaf': 35},
 {'max_depth': 1, 'min_samples_leaf': 40},
 {'max_depth': 1, 'min_samples_leaf': 45},
 {'max_depth': 1, 'min_samples_leaf': 50},
 {'max_depth': 1, 'min_samples_leaf': 55},
 {'max_depth': 1, 'min_samples_leaf': 60},
 {'max_depth': 1, 'min_samples_leaf': 65},
 {'max_depth': 1, 'min_samples_leaf': 70},
 {'max_depth': 1, 'min_samples_leaf': 75},
 {'max_depth': 1, 'min_samples_leaf': 80},
 {'max_depth': 1, 'min_samples_leaf': 85},
 {'max_depth': 1, 'min_samples_leaf': 90},
 {'max_depth': 1, 'min_samples_leaf': 95},
 {'max_depth': 1, 'min_samples_leaf': 100},
 {'max_depth': 2, 'min_samples_leaf': 10},
 {'max_depth': 2, 'min_samples_leaf': 15},
 {'max_depth': 2, 'min_samples_leaf': 20},
 {'max_depth': 2, 'min_samples_leaf': 25},
 {'max_dep

In [48]:
model1_gs.best_params_

{'max_depth': 8, 'min_samples_leaf': 35}

In [49]:
model1_gs.best_score_

0.7008571428571428

In [54]:
pred1 = model1_gs.predict(x_val)

print(confusion_matrix(y_val, pred1))
print('-'*35)
print(classification_report(y_val, pred1))
# best_score_와 classfication_report의 accuracy와 차이가 나는 이유는?

[[498 240]
 [243 519]]
-----------------------------------
              precision    recall  f1-score   support

           0       0.67      0.67      0.67       738
           1       0.68      0.68      0.68       762

    accuracy                           0.68      1500
   macro avg       0.68      0.68      0.68      1500
weighted avg       0.68      0.68      0.68      1500

